In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from app.services.graph import GraphRoute
from app.services.algoritms.a_star import optimized_multi_objective_routing
from app.services.algoritms.dijkstra import dijkstra_multi_objective
from app.services.algoritms.aco import aco_optimized_routing
from app.utils.time import time_to_seconds

In [3]:
graph = GraphRoute(
    origem="Casa da Musica",
    destino="Casino da Póvoa de Varzim, 4490-403",
)

Fusão concluída: 922 paragens no corredor intermodal.
Analisando 2536 pares candidatos para transferência real...


In [4]:
# --- Parâmetros de Roteamento ---
START_TIME = '08:00:00'
start_sec = time_to_seconds(START_TIME)

print("\n--- ROTEAMENTO MULTI-OBJETIVO E TIME-DEPENDENT ---\n")
print(f" * {graph.G.nodes[graph.origem_node_id]['name']} - {graph.origem}")
print(f"    - (lat: {graph.geo_origem.y} lon: {graph.geo_origem.x})")
print(f" * {graph.G.nodes[graph.destino_node_id]['name']} - {graph.destino}")
print(f"    - (lat: {graph.geo_destino.y} lon: {graph.geo_destino.x})\n")
print(f"Início da Viagem: {START_TIME}")


--- ROTEAMENTO MULTI-OBJETIVO E TIME-DEPENDENT ---

 * Ponto de Partida - Casa da Musica
    - (lat: 41.1589025 lon: -8.6307748)
 * Ponto de Chegada - Casino da Póvoa de Varzim, 4490-403
    - (lat: 41.3787241 lon: -8.7661968)

Início da Viagem: 08:00:00


In [5]:
print("\n--- Algoritmo A* ---")

a_star_pareto_solutions = optimized_multi_objective_routing(
    graph.G, graph.origem_node_id, graph.destino_node_id, start_sec
)

print(f"\n✅ Algoritmo concluído. Encontradas {len(a_star_pareto_solutions)} soluções não dominadas (Fronteira de Pareto).")


--- Algoritmo A* ---

✅ Algoritmo concluído. Encontradas 3 soluções não dominadas (Fronteira de Pareto).


In [6]:
print("\n--- Algoritmo ACO ---")

aco_pareto_solutions = aco_optimized_routing(
    graph.G, graph.origem_node_id, graph.destino_node_id, start_sec
)

print(f"\n✅ Algoritmo concluído. Encontradas {len(aco_pareto_solutions)} soluções não dominadas (Fronteira de Pareto).")


--- Algoritmo ACO ---
🚀 Iniciando ACO Agressivo: 30 formigas, 20 gerações...
  Iteração 0: 0 soluções na fronteira.
  Iteração 5: 3 soluções na fronteira.
  Iteração 10: 6 soluções na fronteira.
  Iteração 15: 4 soluções na fronteira.

✅ Algoritmo concluído. Encontradas 4 soluções não dominadas (Fronteira de Pareto).


In [7]:
print("\n--- Algoritmo Dijkstra ---")

dijkstra_pareto_solutions = dijkstra_multi_objective(
    graph.G, graph.origem_node_id, graph.destino_node_id, start_sec
)

print(f"\n✅ Algoritmo concluído. Encontradas {len(dijkstra_pareto_solutions)} soluções não dominadas (Fronteira de Pareto).")


--- Algoritmo Dijkstra ---

✅ Algoritmo concluído. Encontradas 8 soluções não dominadas (Fronteira de Pareto).


In [ ]:
def display_itineraries(solutions):
    if not solutions:
        print("❌ Nenhuma rota encontrada para os critérios selecionados.")
        return

    # Ordenar por tempo para a visualização
    sorted_sols = sorted(solutions, key=lambda s: s.total_time)

    print(f"\n{'='*60}")
    print(f"{'MODO':<12} | {'TEMPO':<10} | {'CO2 (g)':<10} | {'CAMINHADA':<10}")
    print(f"{'-'*60}")

    for i, sol in enumerate(sorted_sols):
        # Identificar o perfil da solução
        profile = "Rápida" if i == 0 else ""
        if sol == min(sorted_sols, key=lambda s: s.total_co2): profile = "Ecológica"
        if sol == max(sorted_sols, key=lambda s: s.total_walk_km): profile = "Ativa"

        mins = int(sol.total_time // 60)
        co2 = round(sol.total_co2, 1)
        walk = round(sol.total_walk_km, 2)
        
        print(f"Opção {i+1:<2} {profile:<10} | {mins:<2} min    | {co2:<10} | {walk:<5} km")

    print(f"{'='*60}\n")

In [ ]:
display_itineraries(a_star_pareto_solutions)

In [ ]:
display_itineraries(aco_pareto_solutions)

In [ ]:
display_itineraries(dijkstra_pareto_solutions)